In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

In [2]:
# Parameters

EPOCHS = 6
BATCH_SIZE = 2

LEARNING_RATE = 0.0001
DROPOUT = 0.75

# Placeholders
#  4D = [batch, img_h, img_w, num_classes]
correct_label = tf.placeholder(tf.float32, [None, 160, 576, 2])
keep_prob = tf.placeholder(tf.float32)
learning_rate = tf.placeholder(tf.float32)

In [3]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


In [4]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out
tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [5]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    # 1x1 convolution of vgg encoder last layer
    conv_1x1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='same', kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))

    # Scaling up by 2 of conv_1x1
    deconv_1 = tf.layers.conv2d_transpose(conv_1x1, num_classes, 4, 2, padding='same',kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    # 1x1 convolution of vgg encoder layer 4 output
    vgg_layer4_conv_1x1 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding='same',kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    # combine the two
    combine_1 = tf.add(deconv_1, vgg_layer4_conv_1x1)
    

    # Scaling up by 2 of combine_1 
    deconv_2 = tf.layers.conv2d_transpose(combine_1, num_classes, 4, 2, padding='same',kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    # 1x1 convolution of vgg encoder layer 3 output
    vgg_layer3_conv_1x1 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding='same',kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    # combine the two
    combine_2 = tf.add(deconv_2, vgg_layer3_conv_1x1)
    
    with tf.name_scope('encode_final_layers'):
        output = tf.layers.conv2d_transpose(combine_2, num_classes, 16, 8, padding='same',kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
        tf.summary.histogram('encode_final_upsample', output)
    
    
    return output
tests.test_layers(layers)

Tests Passed


In [6]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    # Convert 4D input to 2D. Row - pixel, column - class
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    gt_labels = tf.reshape(correct_label, (-1, num_classes))

    # Loss in softmax
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=gt_labels)
    cross_entropy_loss = tf.reduce_mean(cross_entropy)

    # AdamOpt train operator
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)

    
    return logits, train_op, cross_entropy_loss
tests.test_optimize(optimize)

Tests Passed


In [7]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    for epochs in range(EPOCHS):
        losses = []
        i = 0
        
        for images , labels in get_batches_fn(BATCH_SIZE):
            i += 1
            feed = { input_image: images,
                     correct_label: labels,
                     keep_prob: DROPOUT,
                     learning_rate: LEARNING_RATE }
            _, loss = sess.run([train_op,cross_entropy_loss], feed_dict = feed)
            print("iteration: ", i , " current loss:", loss)
            losses.append(loss)
            
        training_loss = sum(losses) / len(losses)
        print("epoch: ", epochs + 1 , " of ", EPOCHS, " training loss: ", training_loss)
    
    
tests.test_train_nn(train_nn)

INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'


In [8]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        image_input, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        layer_output = layers(layer3_out,layer4_out,layer7_out, num_classes)
        logits, train_op, cross_entropy_loss = optimize(layer_output,correct_label, learning_rate, num_classes)

        # TODO: Train NN using the train_nn function
        
        # Initialize all variables
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        
        train_nn(sess, EPOCHS, BATCH_SIZE, get_batches_fn,train_op,cross_entropy_loss, image_input, correct_label, keep_prob,learning_rate)

        # TODO: Save inference data using helper.save_inference_samples
        #  helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape,logits, keep_prob, image_input)
        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    run()

ResourceExhaustedError: OOM when allocating tensor with shape[2,4096,5,18]
	 [[Node: fc6/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](pool5, fc6/weights/read)]]
	 [[Node: Mean/_79 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1709_Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'fc6/Conv2D', defined at:
  File "/home/bryan-eys3d/.conda/envs/carnd-term3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/bryan-eys3d/.conda/envs/carnd-term3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/bryan-eys3d/.conda/envs/carnd-term3/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch